<h3>Toy example experiment 6.18</h3>

Use the magenpy GWASimulator based on RA's genotypes data (d1CM22_sorted.bed) to simulate phenotypes.  
the heritibility = 0.5, (Gaussian Mixture Proportion)pi = [.85, .15]

In [48]:
import numpy as np
import pandas as pd
import magenpy as mgp
import viprs as vp

In [49]:
#simulate 
g_sim = mgp.GWASimulator("CMdream1/chr22/d1CM22_sorted",
                         pi = [.85, .15],
                         h2=0.5)
g_sim.simulate()
g_sim.to_phenotype_table()

> Reading BED file...
> Generating polygenic scores...


,FID,IID,phenotype
0,0,1721000,0.907175
1,0,1722000,0.096401
2,0,1723000,-0.529977
3,0,1725000,-1.965584
4,0,1726000,-1.112442
...,...,...,...
172,0,1921000,-0.885473
173,0,1922000,-0.400395
174,0,1923000,-1.208614
175,0,1924000,-1.140223


In [50]:
pd.read_csv("CMdream1/chr22/d1CM22_sorted.bim", sep="\t", header=None)


,0,1,2,3,4,5
0,22,rs4911642,-11.277499,14884399,G,A
1,22,rs2027653,-7.277181,15298335,G,A
2,22,rs5747620,-6.171966,15412698,G,A
3,22,rs5747968,-5.835598,15447504,C,A
4,22,rs2236639,-5.787480,15452483,A,G
...,...,...,...,...,...,...
7915,22,rs715586,70.074142,49510004,A,G
7916,22,rs8137951,70.076278,49512530,A,G
7917,22,rs756638,70.084762,49518559,A,G
7918,22,rs3810648,70.168330,49522492,G,A


In [51]:
g_sim.perform_gwas()
g_sim.to_summary_statistics_table().to_csv(
    "Toy_example_expr/d1CM22_sorted.sumstats", sep="\t", index=False
)

> Performing GWAS...


In [52]:
# Load summary statistics（simulate phenotype from above) and match them with perviously
gdl_sim = mgp.GWADataLoader(bed_files="CMdream1/chr22/d1CM22_sorted",
                            sumstats_files="Toy_example_expr/d1CM22_sorted.sumstats",
                            sumstats_format="magenpy")

> Reading BED file...
> Reading summary statistics file...
> Harmonizing data...


In [53]:
pd.read_csv("Toy_example_expr/d1CM22_sorted.sumstats", sep="\t", header=None)

,0,1,2,3,4,5,6,7,8,9,10
0,CHR,SNP,POS,A1,A2,MAF,N,BETA,Z,SE,PVAL
1,22,rs4911642,14884399,G,A,0.12721893491124261,169,-0.04265189418220031,-0.5544746243686042,0.07692307692307691,0.5792540674234714
2,22,rs2027653,15298335,G,A,0.3742690058479532,171,0.022917417786659317,0.2996841245368486,0.07647191129018725,0.764418109140301
3,22,rs5747620,15412698,G,A,0.3953488372093023,172,0.014003965819679977,0.18366028991795585,0.07624928516630233,0.8542799629683887
4,22,rs5747968,15447504,C,A,0.3700564971751412,177,0.06738289268960251,0.8964710805250063,0.07516460280028288,0.37000121905585104
...,...,...,...,...,...,...,...,...,...,...,...
7916,22,rs715586,49510004,A,G,0.14204545454545456,176,0.02605969181150253,0.34572087956420366,0.07537783614444091,0.7295524911541165
7917,22,rs8137951,49512530,A,G,0.2768361581920904,177,0.014883396930397884,0.19801071749083826,0.07516460280028288,0.8430366784096935
7918,22,rs756638,49518559,A,G,0.2471590909090909,176,-0.005179443115421703,-0.0687130777473729,0.07537783614444091,0.9452180082723375
7919,22,rs3810648,49522492,G,A,0.048295454545454544,176,-0.03548146288556614,-0.4707147976173748,0.07537783614444091,0.6378444157603079


In [54]:
gdl_sim.compute_ld(estimator='sample',
                   output_dir='Toy_example_expr/out/ld/')

> Computing LD matrix...


Use Viprs, initialize the viprs model based on the <code> gdl_sim model</code> containing the GWAS data and the LD matrix (using xarray method) for the chr22.

In [55]:
v = vp.VIPRS(gdl_sim)

In [56]:
v.fit()

> Initializing model parameters
> Loading LD matrices into memory...
> Performing model fit...
> Using up to 1 threads.


  3%|▎         | 33/1000 [00:05<02:33,  6.32it/s]

Converged at iteration 34 || ELBO: -251.022620
> Final ELBO: -251.022620
> Estimated heritability: 0.063632
> Estimated proportion of causal variants: 0.004769


In [57]:
# Once the model converges, we can generate PRS estimates 
prs_chr22 = v.predict()
print(len(prs_chr22))
prs_chr22

> Generating polygenic scores...
177


array([ 0.02524362,  0.00340651, -0.02926352, -0.08015242, -0.05001765,
       -0.01754011, -0.05930407,  0.04780193, -0.04495885, -0.06588032,
       -0.00294896, -0.04289826, -0.05234844,  0.01234762, -0.03597946,
        0.05572808, -0.00871157, -0.03428585, -0.0189071 , -0.01792462,
       -0.03905429, -0.00810673, -0.07329661,  0.00371513, -0.0502091 ,
        0.01321316,  0.00340567, -0.00755528, -0.00198109,  0.06531484,
       -0.05079751,  0.04293674,  0.07368653,  0.08399345,  0.01447073,
       -0.01233348,  0.03267192,  0.08306909,  0.04610853,  0.05215081,
       -0.04226014, -0.04836083, -0.0148821 ,  0.01901929,  0.00442117,
       -0.05467186, -0.01141341,  0.01739579, -0.0512659 ,  0.00913866,
       -0.00750138, -0.06343894,  0.01013593,  0.0121281 , -0.00382063,
       -0.007742  , -0.00643023, -0.02042649,  0.04363794, -0.09873318,
        0.02001188,  0.04213866,  0.02244945,  0.02269127,  0.0296657 ,
       -0.07756116,  0.05037002, -0.0365212 ,  0.09033644, -0.04

In [58]:
# To examine posterior estimates for the model parameters, you can simply invoke the .to_table() method:
v.to_table()

,CHR,SNP,A1,A2,BETA,PIP,VAR_BETA
0,22,rs4911642,G,A,-0.000040,0.004367,0.000006
1,22,rs2027653,G,A,0.000020,0.004255,0.000005
2,22,rs5747620,G,A,0.000015,0.004237,0.000005
3,22,rs5747968,C,A,0.000063,0.004547,0.000006
4,22,rs2236639,A,G,0.000023,0.004254,0.000005
...,...,...,...,...,...,...,...
7915,22,rs715586,A,G,0.000026,0.004271,0.000005
7916,22,rs8137951,A,G,0.000011,0.004213,0.000005
7917,22,rs756638,A,G,-0.000006,0.004209,0.000005
7918,22,rs3810648,G,A,-0.000030,0.004293,0.000005


<code>Validation</code> use another the simulate phenotype still on the d1chr22_sorted.bed as the validation part. 

In [62]:
#simulate 
g2_sim = mgp.GWASimulator("CMdream1/chr22/d1CM22_sorted",
                         pi = [.85, .15],
                         h2=0.5)
g2_sim.simulate()
a = g2_sim.to_phenotype_table()
a.to_csv("Toy_example_expr/phenotype/g2_sim_pheno.txt",sep='\t', index=False, header=False)
g2_sim.perform_gwas()
g2_sim.to_summary_statistics_table().to_csv(
    "Toy_example_expr/d1CM22_sorted_valid.sumstats", sep="\t", index=False
)

# Load summary statistics（simulate phenotype from above) and match them with perviously
gdl2_sim = mgp.GWADataLoader(bed_files="CMdream1/chr22/d1CM22_sorted",
                            phenotype_file="Toy_example_expr/phenotype/g2_sim_pheno.txt",
                            sumstats_files="Toy_example_expr/d1CM22_sorted_valid.sumstats",
                            sumstats_format="magenpy")
gdl2_sim.compute_ld(estimator='sample',
                   output_dir='Toy_example_expr/out/ld_val/')


> Reading BED file...
> Generating polygenic scores...
> Performing GWAS...
> Reading BED file...
> Reading phenotype file...
> Reading summary statistics file...
> Harmonizing data...
> Computing LD matrix...


Exception ignored in: <finalize object at 0x7f7ad84addc0; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/lib/python3.7/tempfile.py", line 934, in _cleanup
    _rmtree(name)
  File "/usr/lib/python3.7/shutil.py", line 485, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/usr/lib/python3.7/shutil.py", line 483, in rmtree
    orig_st = os.lstat(path)
FileNotFoundError: [Errno 2] No such file or directory: 'temp/ld_27inuwdj'


In [63]:
val_prs = v.predict(gdl2_sim)

> Generating polygenic scores...


In [64]:
from viprs.eval.metrics import r2 
r2(val_prs, gdl2_sim.sample_table.phenotype)

0.00010823635519292879

Validation use the chr22 genotypes in <code> dream2 </code> file sets.  
Do the same steps as before to got <code>"CMdream2/chr22/d2CM22_sorted"</code> binary file sets (.bed + .fam + .bim).  
Sorted the SNPs from dream2 for chromosome 22 and add CM values to it. 

In [65]:
d2_bim = pd.read_csv("data/RA_challenge_qced_genotypes/dream2.bim", sep="\t", header=None)
d2_filter_bim = d2_bim[d2_bim[0] ==22][1]
d2_sorted_chr22 = d2_filter_bim.sort_values()
d2_sorted_chr22.to_csv("CMdream2/d2_sorted_chr22_bim.txt", header=False,
                       index=False, sep="\t")

In [66]:
# Use simulate method to create phenotpyes 
#simulate 
g3_sim = mgp.GWASimulator("CMdream2/chr22/d2CM22_sorted",
                         pi = [.85, .15],
                         h2=0.5)
g3_sim.simulate()
a = g3_sim.to_phenotype_table()
a.to_csv("Toy_example_expr/phenotype/val_d2chr22_pheno.txt",sep='\t', index=False, header=False)
g3_sim.perform_gwas()
g3_sim.to_summary_statistics_table().to_csv(
    "Toy_example_expr/val_d2chr22_valid.sumstats", sep="\t", index=False
)

# Load summary statistics（simulate phenotype from above) and match them with perviously
gdl3_sim = mgp.GWADataLoader(bed_files="CMdream2/chr22/d2CM22_sorted",
                            phenotype_file="Toy_example_expr/phenotype/val_d2chr22_pheno.txt",
                            sumstats_files="Toy_example_expr/val_d2chr22_valid.sumstats",
                            sumstats_format="magenpy")
gdl3_sim.compute_ld(estimator='sample',
                   output_dir='Toy_example_expr/d2chr22_out/ld_val/')

> Reading BED file...
> Generating polygenic scores...
> Performing GWAS...
> Reading BED file...
> Reading phenotype file...
> Reading summary statistics file...
> Harmonizing data...
> Computing LD matrix...


In [67]:
val2_prs = v.predict(gdl3_sim)

> Generating polygenic scores...


In [70]:
#To evaluate the accuracy of the polygenic score, 
# you can use some of the provided metrics with the viprs package. 
# For quantitative traits, it is customary to report the prediction R^2, 
# or the proportion of variance explained by the PRS:
r2(val2_prs, gdl3_sim.sample_table.phenotype)

0.0066335364334810295